In [1]:
pip install langchain google-generativeai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\AutomaC\Desktop\AIP\FinQ&A\myenvironment\Scripts\python.exe -m pip install --upgrade pip


In [6]:
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import time

os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"

file_path = r"C:\Users\AutomaC\Desktop\AIP\RAG system new companies\extracted_sec_text\combined_document.txt"

with open(file_path, "r", encoding="utf-8") as f:
    combined_text = f.read()

text_chunks = combined_text.split("\n\n")

embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

start_time = time.time()

embeddings = embedding_model.embed_documents(text_chunks)

# ✅ Check for mismatched counts
if len(text_chunks) == len(embeddings):
    print(f"✅ Successfully embedded all {len(text_chunks)} text chunks!")
else:
    print(f"⚠️ Mismatch detected! Expected {len(text_chunks)} embeddings, but got {len(embeddings)}.")

# Print first 5 embeddings for verification
for i, (text, embed) in enumerate(zip(text_chunks[:5], embeddings[:5])):
    print(f"\n🔹 Chunk {i+1}: {text[:100]}...")  # Print first 100 characters
    print(f"🟢 Embedding {i+1}: {embed[:5]}...")  # Print first 5 embedding values

end_time = time.time()
total_time = end_time - start_time

print(total_time)


✅ Successfully embedded all 147 text chunks!

🔹 Chunk 1: ID: AMD_Q4 2023.txt
Table of ContentsPART IITEM 1. BUSINESS Cautionary Statement Regarding Forward-L...
🟢 Embedding 1: [0.0069469972513616085, -0.012498626485466957, -0.03598563373088837, 0.0023446690756827593, 0.0678253099322319]...

🔹 Chunk 2: ==================================================...
🟢 Embedding 2: [0.022281521931290627, -0.002733333269134164, -0.07591065764427185, -0.007210069335997105, 0.047469064593315125]...

🔹 Chunk 3: ID: AT&T_Q3 2024.txt
TABLE OF CONTENTSSection 1.1 The Debt-Financed Distribution 2Section 1.2 INVIDI...
🟢 Embedding 3: [-0.011203193105757236, 0.006782775279134512, -0.03247274085879326, -0.011494665406644344, 0.03130218759179115]...

🔹 Chunk 4: ==================================================...
🟢 Embedding 4: [0.022281521931290627, -0.002733333269134164, -0.07591065764427185, -0.007210069335997105, 0.047469064593315125]...

🔹 Chunk 5: ID: AT&T_Q4 2023.txt
TABLE OF CONTENTSPART IITEM 1. BUSIN

In [8]:
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_by_headings(text):
    """Chunk document by headings (H1, H2, etc.)."""
    chunks = re.split(r"\n\s*(?=[A-Z][A-Za-z ]{3,}\n)", text)  # Split at uppercase headings
    return [chunk.strip() for chunk in chunks if chunk.strip()]

def recursive_chunking(chunks, chunk_size=1000, chunk_overlap=200):
    """Apply recursive chunking after heading-based split."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    refined_chunks = []
    for chunk in chunks:
        refined_chunks.extend(text_splitter.split_text(chunk))
    return refined_chunks

# Load the document
file_path = r"C:\Users\AutomaC\Desktop\AIP\RAG system new companies\extracted_sec_text\combined_document.txt"
with open(file_path, "r", encoding="utf-8") as f:
    document_text = f.read()

# Step 1: Split by headings
heading_chunks = split_by_headings(document_text)

# Step 2: Apply Recursive Chunking
final_chunks = recursive_chunking(heading_chunks)

# Save output
output_path = r"C:\Users\AutomaC\Desktop\AIP\RAG system new companies\final_chunked_document.json"
import json
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(final_chunks, f, indent=4)

print(f"✅ Successfully chunked document into {len(final_chunks)} chunks! Saved to {output_path}")


✅ Successfully chunked document into 20978 chunks! Saved to C:\Users\AutomaC\Desktop\AIP\RAG system new companies\final_chunked_document.json


In [1]:
import os
import re
import json
import faiss
import numpy as np
import time
import ollama
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document

# ✅ Set API Key
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"

# ✅ Load Document
file_path = r"C:\Users\AutomaC\Desktop\AIP\RAG system new companies\extracted_sec_text\combined_document.txt"
with open(file_path, "r", encoding="utf-8") as f:
    document_text = f.read()

# ✅ 1️⃣ STEP 1: Chunking (Heading-Based + Recursive Character-Based)
def split_by_headings(text):
    """Split document by headings (H1, H2, etc.)."""
    chunks = re.split(r"\n\s*(?=[A-Z][A-Za-z ]{3,}\n)", text)  # Split at uppercase headings
    return [chunk.strip() for chunk in chunks if chunk.strip()]

def recursive_chunking(chunks, chunk_size=1000, chunk_overlap=200):
    """Apply recursive chunking after heading-based split."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    refined_chunks = []
    for chunk in chunks:
        refined_chunks.extend(text_splitter.split_text(chunk))
    return refined_chunks

# Apply chunking
heading_chunks = split_by_headings(document_text)
final_chunks = recursive_chunking(heading_chunks)

print(f"✅ Chunking completed. Total Chunks: {len(final_chunks)}")

# ✅ 2️⃣ STEP 2: Generate Embeddings Using Google AI
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

start_time = time.time()
embeddings = embedding_model.embed_documents(final_chunks)
end_time = time.time()

print(f"✅ Embeddings generated for {len(final_chunks)} chunks in {round(end_time - start_time, 2)} seconds.")

# ✅ 3️⃣ STEP 3: Store Embeddings in FAISS
embeddings_array = np.array(embeddings).astype("float32")
dimension = embeddings_array.shape[1]  # Get embedding dimensions

# Initialize FAISS Index
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_array)

# Save FAISS index
faiss.write_index(index, r"C:\Users\AutomaC\Desktop\AIP\RAG system new companies\extracted_sec_text\faiss_index")
with open(r"C:\Users\AutomaC\Desktop\AIP\RAG system new companies\extracted_sec_text\faiss_chunks.json", "w", encoding="utf-8") as f:
    json.dump(final_chunks, f, indent=4)

print(f"✅ FAISS index stored with {index.ntotal} embeddings!")

# ✅ 4️⃣ STEP 4: Load FAISS & Ollama for Querying
index = faiss.read_index(r"C:\Users\AutomaC\Desktop\AIP\RAG system new companies\extracted_sec_text\faiss_index")
with open(r"C:\Users\AutomaC\Desktop\AIP\RAG system new companies\extracted_sec_text\faiss_chunks.json", "r", encoding="utf-8") as f:
    stored_chunks = json.load(f)

# Convert stored chunks into LangChain Document objects
docs = [Document(page_content=chunk) for chunk in stored_chunks]

# Convert FAISS into a LangChain retriever
faiss_vector_db = FAISS(embedding_function=embedding_model.embed_query, index=index, texts=[doc.page_content for doc in docs])
retriever = faiss_vector_db.as_retriever()

✅ Chunking completed. Total Chunks: 20978
✅ Embeddings generated for 20978 chunks in 525.94 seconds.


RuntimeError: Error in __cdecl faiss::FileIOWriter::FileIOWriter(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:102: Error: 'f' failed: could not open C:\Users\AutomaC\Desktop\AIP\RAG system new companies\extracted_sec_text\faiss_index for writing: Permission denied

In [16]:
# ✅ 4️⃣ STEP 4: Load FAISS & Ollama for Querying
index = faiss.read_index(r"C:\Users\AutomaC\Desktop\AIP\RAG system new companies\faiss_index")
with open(r"C:\Users\AutomaC\Desktop\AIP\RAG system new companies\extracted_sec_text\faiss_chunks.json", "r", encoding="utf-8") as f:
    stored_chunks = json.load(f)

# Convert stored chunks into LangChain Document objects
docs = [Document(page_content=chunk) for chunk in stored_chunks]

# ✅ Create FAISS vector store from texts
faiss_vector_db = FAISS.from_texts(
    [doc.page_content for doc in docs],  # Text content for FAISS storage
    embedding_model,  # Embedding function
)

# ✅ Save FAISS Index (Persistence)
faiss_vector_db.save_local(r"C:\Users\AutomaC\Desktop\AIP\RAG system new companies\extracted_sec_text\faiss_index")

# ✅ Load the FAISS Index for Retrieval
faiss_vector_db = FAISS.load_local(r"C:\Users\AutomaC\Desktop\AIP\RAG system new companies\extracted_sec_text\faiss_index", embedding_model)

# ✅ Create the retriever
retriever = faiss_vector_db.as_retriever()

ValueError: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).

In [29]:
# ✅ Load the FAISS Index for Retrieval
faiss_vector_db = FAISS.load_local(r"C:\Users\AutomaC\Desktop\AIP\RAG system new companies\extracted_sec_text\faiss_index", embedding_model, allow_dangerous_deserialization=True)

# ✅ Create the retriever
retriever = faiss_vector_db.as_retriever()

 retriever

In [30]:
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000026ED6CD2510>, search_kwargs={})

In [14]:
import os
import faiss
import numpy as np
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_groq import ChatGroq  # ✅ Use ChatGroq for API integration

# ✅ Set API Key for Groq
os.environ["GROQ_API_KEY"] = "GROQ_API_KEY"

# ✅ Load Groq Model via LangChain
llm = ChatGroq(model_name="gemma2-9b-it")  

# ✅ Function to Query FAISS and Generate Response using Groq
def query_groq(question):
    # Retrieve relevant chunks from FAISS
    relevant_docs = retriever.get_relevant_documents(question)
    retrieved_text = "\n\n".join([doc.page_content for doc in relevant_docs])

    # ✅ Construct query using LangChain LLM (No API URL required)
    response = llm.invoke(f"Based on the following retrieved information, answer the question:\n\n{retrieved_text}\n\nQuestion: {question}")

    return response

# ✅ Example Query
user_question = "What are the primary segments that AMD operates in?"
response = query_groq(user_question)
print("\n🔹 Response from Groq:\n", response)

InternalServerError: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}}

In [17]:
# ✅ Function to Query FAISS and Use Ollama for Response
def query_ollama(question, model="qwen:1.8b"):
    """
    Retrieves relevant chunks from FAISS and queries Ollama LLM.
    
    Args:
    - question (str): User question.
    - model (str): LLM model to use (e.g., "mistral", "gemma", "llama3").

    Returns:
    - str: LLM-generated answer.
    """


    # ✅ Query Ollama LLM
    response = ollama.chat(
        model=model,
        messages=[
            {"role": "system", "content": "You are an financial AI assistant that provides concise answers based on retrieved documents."},
            {"role": "user", "content": f"Based on the following retrieved information, answer the question:\n\n{retrieved_text}\n\nQuestion: {question}"}
        ]
    )

    return response["message"]["content"]

In [22]:
user_question = input("What Question you needs to ask?")
response = query_ollama(user_question, model="qwen:1.8b")  # Change model if needed
print("\n🔹 Response from Ollama:\n", response)

What Question you needs to ask?What is AMD's primary business focus?

🔹 Response from Ollama:
 AMD's primary business focus is serving the global consumer market for custom-designed high-performance computing (HPC) solutions and products. The company has a diverse portfolio of HPC products and services, including Ryzen processors, EPYC servers, GPUs, FPGAs, smartNICs, AI accelerators, and Adaptive SoC products.

AMD's primary business focus is centered around providing customized HPC solutions that meet the specific needs and requirements of its clients. The company has a proven track record of delivering high-quality custom-designed HPC solutions to its clients, thereby contributing significantly to their operations and increasing their overall competitiveness in the global consumer market for custom-designed HPC solutions


In [23]:
user_question = input("What Question you needs to ask?")
response = query_ollama(user_question, model="qwen:1.8b")  # Change model if needed
print("\n🔹 Response from Ollama:\n", response)

What Question you needs to ask?What are AMD’s four primary business segments?

🔹 Response from Ollama:
 AMD's four primary business segments are as follows:

1. Data Center Graphics Segment:
   - This segment focuses on providing high-performance graphics solutions for data center environments, such as servers, routers, switches, and storage systems.
   - The products offered in this segment include AMD Radeon Pro graphics chips, AMD EPYC processors, AMD Xe server blades, and other related graphics cards and systems.

2. Server Micrographics Segment:
   - This segment focuses on providing a wide range of high-performance micrograph solutions for both server and embedded applications, such as notebooks, laptops, desktop computers, embedded devices, IoT devices, smart edge devices, drones, autonomous vehicles, and other related micrograph systems and products.
   
3. Test and Measurement Graphics Segment:
   - This segment focuses on providing a wide range of high-performance test and me

In [24]:
user_question = input("What Question you needs to ask?")
response = query_ollama(user_question, model="qwen:1.8b")  # Change model if needed
print("\n🔹 Response from Ollama:\n", response)

What Question you needs to ask?What products does AMD offer for the data center market?

🔹 Response from Ollama:
 AMD offers a wide range of products for the data center market. Some of the key products offered by AMD include:

1. CPUs (Central Processing Units): 
   AMD offers a diverse range of CPUs for the data center market. Some of the key CPU models offered by AMD include:

  * AMD EPYC™ processors:
     AMD EPYC™ processors are designed to deliver high performance, energy efficiency, and scalability in the data center market. Some of the key features and capabilities of AMD EPYC™ processors include:

  * High-performance CPU architecture: 
     AMD EPYC™ processors feature a high-performance CPU (CPU) architecture that enables efficient execution of operations and tasks across the data center market.

  * Advanced energy efficiency design: 
     AMD EPYC™ processors are designed to deliver advanced energy efficiency in the data center market. Some of the key features and capabil

In [25]:
user_question = input("What Question you needs to ask?")
response = query_ollama(user_question, model="qwen:1.8b")  # Change model if needed
print("\n🔹 Response from Ollama:\n", response)

What Question you needs to ask?What offerings does Google Cloud provide?

🔹 Response from Ollama:
 Google Cloud provides a wide range of cloud-based services and offerings across various categories such:

  1. **Infrastructure-as-a-Service (IaaS)**:** This category offers on-demand computing resources, including virtual machines, storage, and network connectivity services, enabling customers to rent or purchase computing resources as needed without the need for upfront capital investments.
    
  2. **Platform-as-a--Service (PaaS)**:** This category offers software platforms that allow developers to build, deploy, and manage applications and other software services on a serverless architecture, enabling customers to focus on application development and deployment rather than worrying about infrastructure management and costs.
    
  3. **Cloud Functions**: This category offers compute services in the form of virtual machines and other computing resources that can be invoked by APIs or 

In [26]:
user_question = input("What Question you needs to ask?")
response = query_ollama(user_question, model="qwen:1.8b")  # Change model if needed
print("\n🔹 Response from Ollama:\n", response)

What Question you needs to ask?How does Google protect user data and privacy?

🔹 Response from Ollama:
 Google protects user data and privacy through a combination of technical measures and legal compliance.
1. Technical Measures:
   - Data Encryption: Google uses encryption algorithms like AES (Advanced Encryption Standard) to protect user data in transit. This ensures that even if an attacker intercepts the encrypted data during transmission, they will not be able to read or access the original data without first decrypting it using the same encryption algorithm used to encrypt the data.
   - Data Access Controls: Google has implemented strict data access controls (DACs)) to ensure that only authorized personnel have access to sensitive user data. This involves implementing role-based access control (RBAC) policies, which specify who can perform specific actions within a particular resource scope or permission level. This ensures that even if an attacker attempts to gain unauthorized

In [27]:
user_question = input("What Question you needs to ask?")
response = query_ollama(user_question, model="qwen:1.8b")  # Change model if needed
print("\n🔹 Response from Ollama:\n", response)

What Question you needs to ask?What legal and regulatory risks does Google face?

🔹 Response from Ollama:
 Google faces several legal and regulatory risks, which can have significant implications for the company's business operations, competitive position, financial health, and overall brand reputation.

1. Regulatory Compliance Risks:

a) Federal Trade Commission (FTC): The FTC is responsible for enforcing antitrust laws, including those related to Google's advertising practices. In 2018, the FTC launched an investigation into Google's advertising practices, including its use of ad blockers, and whether it was engaging in anti-competitive behavior. The FTC found that Google was using ad blockers to reduce the amount of money paid for search engine advertising, and that Google was also engaging in anti-competitive behavior by using its dominant position in the online advertising market to restrict access to its ad inventory, which could limit the number of ads displayed on Google's sea

In [28]:
user_question = input("What Question you needs to ask?")
response = query_ollama(user_question, model="qwen:1.8b")  # Change model if needed
print("\n🔹 Response from Ollama:\n", response)

What Question you needs to ask?How does Google’s advertising model work?

🔹 Response from Ollama:
 Google's advertising model works through a combination of revenue models, targeting and optimization strategies, and content management systems.

1. Revenue Models:
    - AdWords revenue model: This model generates revenue when users click on ads displayed in the context of search queries.
    - Google Display Network (GDN) revenue model: This model generates revenue when users view ads displayed on third-party websites participating in the GDN network.
    - YouTube revenue model: This model generates revenue when a user interacts with a YouTube video, such as watching or sharing a video. The revenue generated from these interactions is recognized in the relevant financial statements.

2. Targeting and Optimization Strategies:
    - Keyword targeting: Google's advertising platform allows advertisers to target specific keywords that are likely to appear in search queries.
    - Geographic

In [ ]:
# ✅ Function to Query FAISS and Use Ollama for Response
def query_ollama(question, model="qwen:1.8b"):
    """
    Retrieves relevant chunks from FAISS and queries Ollama LLM.
    
    Args:
    - question (str): User question.
    - model (str): LLM model to use (e.g., "mistral", "gemma", "llama3").

    Returns:
    - str: LLM-generated answer.
    """


    # ✅ Query Ollama LLM
    response = ollama.chat(
        model=model,
        messages=[
            {"role": "system", "content": "You are an financial AI assistant that provides concise answers based on retrieved documents."},
            {"role": "user", "content": f"Based on the following retrieved information, answer the question:\n\n{retrieved_text}\n\nQuestion: {question}"}
        ]
    )

    return response["message"]["content"]

In [70]:
import os
from groq import Groq

# ✅ Set API Key for Groq
GROQ_API_KEY = "GROQ_API_KEY"

def query_llm_groq(question):
    """Queries Groq API (gemma2-9b-it model) and returns the response."""

    try:
        # Retrieve relevant chunks from FAISS
        relevant_docs = retriever.get_relevant_documents(question)
        retrieved_text = "\n\n".join([doc.page_content for doc in relevant_docs])
        
        # ✅ Initialize Groq API client
        client = Groq(api_key=GROQ_API_KEY)

        # ✅ Construct API request
        response = client.chat.completions.create(
            model="mixtral-8x7b-32768",
            messages=
                 [{
            "role": "user",
            "content": f"You are an financial AI assistant that provides concise answers based on retrieved documents. Based on the following retrieved information, answer the question:\n\n{retrieved_text}\n\nQuestion: {question}"
                  }],
            temperature=0.3,
            max_tokens=512,
            top_p=1,
            stream=False,
        )

            
        # ✅ Extract and return response content
        return response.choices[0].message.content

    except Exception as e:
        return f"❌ Groq API Error: {str(e)}"

In [73]:
user_question = input("\n💬 Enter your financial question: ")
response = query_llm_groq(user_question)
print("\n🔹 Response from Groq:\n", response)


💬 Enter your financial question: How much worth is to invest in google?

🔹 Response from Groq:
 The value of investing in Google (Alphabet Inc.) depends on various factors, including the investor's financial goals, risk tolerance, and market conditions. As of December 31, 2023, there were approximately 7,305 and 1,757 stockholders of record of Google's Class A and Class C stock, respectively. Google has never declared or paid any cash dividend on its common or capital stock. The company's primary use of capital is for headcount, particularly due to annual stock-based compensation awards that generally vest over four years. Cost of revenues is comprised of Traffic Acquisition Costs (TAC) and other costs of revenues, including compensation expenses related to data centers and other operations, content acquisition costs, depreciation expense, and inventory.

Google is investing significantly in its Research and Development (R&D) efforts, including metaverse and wearables, which may not g